In [ ]:
import requests, pprint, json

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
a = "sk-or-v1-7995770d42"
b = "78c5cbb2d0ab4335adf3"
c = "b69dfff451870ce8"
d = "9f73cf8b7a9544ce02"

OPENROUTER_API_KEY = a+b+c+d
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"




system_prompt = "You are not a helpfull assistant. you always provide joke answer. you always provide wrong answer."
user_prompt = "Siapakah presiden amerika serikat? buat essay 1000 kata tentang dia"


header = {"Authorization": f"Bearer {OPENROUTER_API_KEY}","Content-Type": "application/json"}
payload = {"model":"x-ai/grok-4-fast", "messages":[
    {"role":"system", "content":system_prompt},
    {"role":"user", "content":user_prompt},
], "stream":True}


# response = requests.post(OPENROUTER_BASE_URL+"/chat/completions", headers=header, json=payload, stream=True)
# response_json = response.json()
# response_message = response_json["choices"][0]["message"]["content"]
# pprint.pprint(response_json)
# print(response_message)

response = requests.post(f"{OPENROUTER_BASE_URL}/chat/completions", headers=header, json=payload, stream=True)

# print (response.json()['choices'][0]['message']['content'])
response.raise_for_status()
for chunk in response.iter_lines():
    if chunk:
        data = chunk.decode('utf-8')
        if data.startswith('data: '):
            data = data[6:]
            if data.strip() != '[DONE]':
                data = json.loads(data)
                chunk = (data['choices'][0]['delta']['content'])
                if chunk:
                    print (chunk, end='', flush=True)


In [ ]:
import requests, json

a = "sk-or-v1-7995770d42"
b = "78c5cbb2d0ab4335adf3"
c = "b69dfff451870ce8"
d = "9f73cf8b7a9544ce02"

OPENROUTER_API_KEY = a+b+c+d
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"



class LLM_API:
    def __init__(self):
        self.model = "x-ai/grok-4-fast"
        self.headers = {"Authorization": f"Bearer {OPENROUTER_API_KEY}","Content-Type": "application/json"}
        self.payload = {"model":self.model, "messages":[], "stream":True}
    
    def set_system_prompt(self, system_prompt: str):
        self.payload['messages'].append({"role":"system", "content":system_prompt})
    
    def add_message(self, role: str, content: str):
        self.payload['messages'].append({"role":role, "content":content})
    
    def clear_messages(self):
        self.payload['messages'] = []
    
    def get_response(self):
        response_text = ""
        response = requests.post(f"{OPENROUTER_BASE_URL}/chat/completions", headers=self.headers, json=self.payload, stream=True)
        # print(self.payload)
        response.raise_for_status()
        for chunk in response.iter_lines():
            if chunk:
                data = chunk.decode('utf-8')    
                if data.startswith('data: '):
                    data = data[6:]
                    if data.strip() != '[DONE]':
                        data = json.loads(data)
                        chunk = (data['choices'][0]['delta']['content'])
                        if chunk:
                            print (chunk, end='', flush=True)
                            response_text += chunk
        return response_text
    def start_chat(self):
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["q", "quit", "exit"]:
                break
            self.add_message("user", user_input)
            response = self.get_response()
            # print("\nAssistant: " + response)
            self.clear_messages()






llm_api = LLM_API()
llm_api.set_system_prompt("You are a helpfull assistant. always follo the user instruction.")





In [ ]:
with open("companies.txt", "r") as f:
    companies = f.read()
# print(companies)
json_schema = """
{
    "name": "string",
    "valuation": "number",
    "country": "string"
}
"""

prompt = f"Parse the following company data into json format. {companies}. Parse it with the following json schema: {json_schema}. Return the json only, no other text or explanation. use bilion dollar as the unit of valuation."
llm_api.add_message("user", prompt)

response = llm_api.get_response()

In [ ]:
respone_json = json.loads(response)

response_df = pd.DataFrame(respone_json)

# print(respone_json[0:10])
# print (response_df[0:10]["valuation"])


plt.figure(figsize=(10, 35))
plt.barh(response_df["name"], response_df["valuation"])
plt.show()



In [ ]:
# with open("companies_json.json", "w") as f:
#     json.dump(respone_json, f)


In [45]:
with open("companies_json.json", "r") as f:
    companies_json = json.load(f)

companies_df = pd.DataFrame(companies_json)


# print(companies_df[0:10])


def company_where_maximum_valuation():
    max_valuation = companies_df["valuation"].max()
    company_name = companies_df[companies_df["valuation"] == max_valuation]["name"].values[0]
    return str(company_name)

def top_company_country(country: str, top_n: int):
    top_n_companies = companies_df[companies_df["country"] == country].sort_values(by="valuation", ascending=False).head(top_n)
    return str(top_n_companies)

def show_unique_country():
    unique_country = companies_df["country"].unique()
    return str(unique_country)

# print("max val company", company_where_maximum_valuation())
# print (top_company_country("USA", 10))
# print(show_unique_country())

tool_schema = [{
    "type": "function",
    "function": {
        "name": "company_where_maximum_valuation",
        "description": "Get the company with the maximum valuation",
    }
}, {
    "type": "function",
    "function": {
        "name": "top_company_country",
        "description": "Get the top n companies in a country",
        "parameters": {
            "type": "object",
            "properties": {
                "country": {"type": "string"},
                "top_n": {"type": "integer"}
            }
        }
}}, {
    "type": "function",
    "function": {
        "name": "show_unique_country",
        "description": "Get the unique countries in the dataset",
    }
}]

tool_dictionary = {
    "company_where_maximum_valuation": company_where_maximum_valuation,
    "top_company_country": top_company_country,
    "show_unique_country": show_unique_country
}


In [ ]:
import requests, json

a = "sk-or-v1-7995770d42"
b = "78c5cbb2d0ab4335adf3"
c = "b69dfff451870ce8"
d = "9f73cf8b7a9544ce02"

OPENROUTER_API_KEY = a+b+c+d
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"



class LLM_API_AGENT:
    def __init__(self):
        self.model = "x-ai/grok-4-fast"
        self.headers = {"Authorization": f"Bearer {OPENROUTER_API_KEY}","Content-Type": "application/json"}
        self.payload = {"model":self.model, "messages":[], "tools":tool_schema, "tool_choice": "auto", "stream":True}
    
    def set_system_prompt(self, system_prompt: str):
        self.payload['messages'].append({"role":"system", "content":system_prompt})
    
    def add_message(self, role: str, content: str):
        self.payload['messages'].append({"role":role, "content":content})
    
    def clear_messages(self):
        self.payload['messages'] = []

    def add_tool_call(self, tool_call_id: str, tool_call_response:str):
        self.payload['messages'].append({
            "role": "tool",
            "content": tool_call_response,
            "tool_call_id": tool_call_id
        })
    
    def get_response(self):
        response_text = ""
        response = requests.post(f"{OPENROUTER_BASE_URL}/chat/completions", headers=self.headers, json=self.payload, stream=True)
        # print(self.payload)
        tool_calls = []
        response.raise_for_status()
        for chunk in response.iter_lines():
            if chunk:
                data = chunk.decode('utf-8')    
                if data.startswith('data: '):
                    data = data[6:]
                    if data.strip() != '[DONE]':
                        # print(data)
                        data = json.loads(data)
                        chunk = (data['choices'][0]['delta']['content'])
                        if chunk:
                            print (chunk, end='', flush=True)
                            response_text += chunk
                        if (data['choices'][0]['delta'].get('tool_calls')):
                            # print (data['choices'][0]['delta']['tool_calls'])
                            tool_call = data['choices'][0]['delta']['tool_calls']
                            for tool in tool_call:
                                arg = json.loads(tool['function']['arguments'])
                                tool_calls.append({
                                    "tool_call_id": tool['id'],
                                    "name": tool['function']['name'],
                                    "arguments": arg
                                })
                            # print(tool_calls)

        return response_text, tool_calls
    def start_chat(self):
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["q", "quit", "exit"]:
                break
            self.add_message("user", user_input)
            response = self.get_response()
            # print("\nAssistant: " + response)
            self.clear_messages()

llm_api = LLM_API_AGENT()
llm_api.set_system_prompt("You are a helpfull assistant. always follow the user instruction. You have access to unique dtabase, use available tools to answer the user question.")

llm_api.add_message("user", "list unique country in the data")

response, tool_calls = llm_api.get_response()

print("AI is calling tools:", tool_calls)

for tool in (tool_calls):
    result = tool_dictionary[tool['name']](**tool['arguments'])
    print("tool execution result:", result)



AI is calling tools: [{'tool_call_id': 'call_43708638', 'name': 'show_unique_country', 'arguments': {}}]
tool execution result: ['USA' 'S. Arabia' 'Taiwan' 'China' 'S. Korea' 'Netherlands' 'France'
 'Switzerland' 'Germany' 'UK' 'Japan' 'UAE' 'India' 'Denmark' 'Canada']


['USA' 'S. Arabia' 'Taiwan' 'China' 'S. Korea' 'Netherlands' 'France'
 'Switzerland' 'Germany' 'UK' 'Japan' 'UAE' 'India' 'Denmark' 'Canada']
